<a href="https://colab.research.google.com/github/amontenegrot/Vowels-project/blob/main/CNN_AMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Permisos y Librerias

Permisos e instalación

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Librerias

In [68]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras import backend as K
#K.set_image_dim_ordering('th')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Conv2D #Solució de error en el nuevo keras
from keras.optimizers import SGD,RMSprop

In [69]:
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

Funciones

In [70]:
def extractor_nombres(ruta):
  lista = []

  for dirName, subdirList, fileList in os.walk(ruta):
    for fname in fileList:
        lista.append(fname) #Añadir el nombre del archivo a la lista
  return lista

#Datos

##Lectura de datos

In [71]:
#Ruta de lectura
PATH_ROOT = '/content/drive/My Drive/Proyectos/Machine learning/CNN Espectrogramas/Datos/Prueba de concepto/Imágenes raw/'

#Carpetas de entrenamiento
TRAIN = PATH_ROOT + 'Train/'
TEST = PATH_ROOT + 'Test/'

#Vocales de entrenamiento 
TRAIN_VOWELS_I = TRAIN + 'i/'
TRAIN_VOWELS_E = TRAIN + 'e/'
TRAIN_VOWELS_A = TRAIN + 'a/'
TRAIN_VOWELS_O = TRAIN + 'o/'
TRAIN_VOWELS_U = TRAIN + 'u/'

#Vocales de prueba
TEST_VOWELS_I = TEST + 'i/'
TEST_VOWELS_E = TEST + 'e/'
TEST_VOWELS_A = TEST + 'a/'
TEST_VOWELS_O = TEST + 'o/'
TEST_VOWELS_U = TEST + 'u/'

#Ruta de salida
PATH_OUT = '/content/drive/My Drive/Proyectos/Machine learning/CNN Espectrogramas/Datos/Prueba de concepto/Alejandro/'

In [72]:
#Declaración de listas con archivos
ies_train = ees_train = aes_train = oes_train = ues_train = []
ies_test = ees_test = aes_test = oes_test = ues_test = []

In [73]:
ies_train = extractor_nombres(TRAIN_VOWELS_I)
ees_train = extractor_nombres(TRAIN_VOWELS_E)
aes_train = extractor_nombres(TRAIN_VOWELS_A)
oes_train = extractor_nombres(TRAIN_VOWELS_O)
ues_train = extractor_nombres(TRAIN_VOWELS_U)

ies_test = extractor_nombres(TEST_VOWELS_I)
ees_test = extractor_nombres(TEST_VOWELS_E)
aes_test = extractor_nombres(TEST_VOWELS_A)
oes_test = extractor_nombres(TEST_VOWELS_O)
ues_test = extractor_nombres(TEST_VOWELS_U)

##Reporte de datos

Conociendo el número de vocales que servirán para alimentar la *red neuronal artificial convolucional*

In [74]:
print('Total de archivos a procesar según la vocal (entrenamiento):', '\n')
print('\t', '/i/:', len(ies_train), 'elementos')
print('\t', '/e/:', len(ees_train), 'elementos')
print('\t', '/a/:', len(aes_train), 'elementos')
print('\t', '/o/:', len(oes_train), 'elementos')
print('\t', '/u/:', len(ues_train), 'elementos', '\n')
print('Total de archivos:', 
      len(ies_train)+len(ees_train)+len(aes_train)+len(oes_train)+len(ues_train), 
      'elementos')

Total de archivos a procesar según la vocal (entrenamiento): 

	 /i/: 145 elementos
	 /e/: 189 elementos
	 /a/: 224 elementos
	 /o/: 157 elementos
	 /u/: 133 elementos 

Total de archivos: 848 elementos


In [75]:
print('Total de archivos a procesar según la vocal (pruebas):', '\n')
print('\t', '/i/:', len(ies_test), 'elementos')
print('\t', '/e/:', len(ees_test), 'elementos')
print('\t', '/a/:', len(aes_test), 'elementos')
print('\t', '/o/:', len(oes_test), 'elementos')
print('\t', '/u/:', len(ues_test), 'elementos', '\n')
print('Total de archivos:', 
      len(ies_test)+len(ees_test)+len(aes_test)+len(oes_test)+len(ues_test), 
      'elementos')

Total de archivos a procesar según la vocal (pruebas): 

	 /i/: 61 elementos
	 /e/: 80 elementos
	 /a/: 95 elementos
	 /o/: 67 elementos
	 /u/: 56 elementos 

Total de archivos: 359 elementos


In [76]:
print('Total de archivos a procesar según la vocal:', '\n')
print('\t', '/i/:', len(ies_train)+len(ies_test), 'elementos')
print('\t', '/e/:', len(ees_train)+len(ees_test), 'elementos')
print('\t', '/a/:', len(aes_train)+len(aes_test), 'elementos')
print('\t', '/o/:', len(oes_train)+len(oes_test), 'elementos')
print('\t', '/u/:', len(ues_test)+len(ues_test), 'elementos', '\n')
print('Total de archivos:',
      len(ies_test)+len(ees_test)+len(aes_test)+len(oes_test)+len(ues_test)+
      len(ies_train)+len(ees_train)+len(aes_train)+len(oes_train)+len(ues_train),
      'elementos')

Total de archivos a procesar según la vocal: 

	 /i/: 206 elementos
	 /e/: 269 elementos
	 /a/: 319 elementos
	 /o/: 224 elementos
	 /u/: 112 elementos 

Total de archivos: 1207 elementos


##Carga de datos

Información en esta [página web](https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/)

In [77]:
#Declaración de variables

batch_size = 10

WIDTH_SIZE = 124  # Ancho en pixeles de la imagen
HEIGHT_SIZE = 734  # Alto en pixeles de la imagen

In [78]:
#Crear el generador de datos
datagen_train = ImageDataGenerator(rescale=1.0/255)
datagen_test = ImageDataGenerator(rescale=1.0/255)

In [79]:
#Iterar generador en el conjunto de datos
train_data = datagen_train.flow_from_directory(TRAIN, class_mode='categorical',
                                         batch_size=batch_size,
                                         target_size=(HEIGHT_SIZE, WIDTH_SIZE))

test_data = datagen_test.flow_from_directory(TEST, class_mode='categorical',
                                        batch_size = batch_size,
                                        target_size=(HEIGHT_SIZE, WIDTH_SIZE))

Found 848 images belonging to 5 classes.
Found 359 images belonging to 5 classes.


#Preprocesamiento de datos

##Formato *one-hot*

Más información en esta [página web](https://www.interactivechaos.com/python/scenario/codificacion-one-hot-encoding-de-un-conjunto-de-caracteristicas-categoricas-con)

In [80]:
vowels_output = ['i', 'e', 'a', 'o', 'u']

ohe = OneHotEncoder(sparse = False)
le = LabelEncoder()

vowels_one_hot = ohe.fit_transform(le.fit_transform(vowels_output).reshape(-1, 1))
nclases = 5

vowels_one_hot

array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

#Red Neuronal Artificial Convolucional

##Creación del modelo

In [81]:
# Contenedor del modelo
modelo = Sequential()

In [82]:
# CONV1 Y MAX-POOLING1
modelo.add(Conv2D(filters=6, kernel_size=(3,3), activation='relu', input_shape=(WIDTH_SIZE,HEIGHT_SIZE,3)))  # Ancho, alto, color (canales)
modelo.add(MaxPooling2D(pool_size=(2,2)))

# CONV2 Y MAX-POOLING2
modelo.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))

# Aplanar, FC1, FC2 y salida
modelo.add(Flatten())
modelo.add(Dense(120,activation='relu'))
modelo.add(Dense(84,activation='relu'))
modelo.add(Dense(nclases,activation='softmax'))

##Compilación del modelo

In [83]:
# Optimizador a usar (gradiente descendente, tasa de aprendizaje = 0.1)
modelo.compile(loss='categorical_crossentropy', 
               optimizer='adam',
               metrics=['accuracy'])

In [84]:
modelo.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 122, 732, 6)       168       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 61, 366, 6)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 59, 364, 16)       880       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 29, 182, 16)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 84448)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 120)               10133880  
_________________________________________________________________
dense_10 (Dense)             (None, 84)               

##Entrenamiento

In [85]:
modelo.fit(train_data,
           epochs=10, 
           validation_data=test_data)

Epoch 1/10
85/85 [==============================] - 8s 88ms/step - loss: 2.0187 - accuracy: 0.4390 - val_loss: 0.7847 - val_accuracy: 0.6825
Epoch 2/10
85/85 [==============================] - 7s 86ms/step - loss: 0.6588 - accuracy: 0.7527 - val_loss: 0.8389 - val_accuracy: 0.6435
Epoch 3/10
85/85 [==============================] - 7s 85ms/step - loss: 0.4970 - accuracy: 0.8167 - val_loss: 0.7735 - val_accuracy: 0.6880
Epoch 4/10
85/85 [==============================] - 7s 85ms/step - loss: 0.3157 - accuracy: 0.8857 - val_loss: 0.8400 - val_accuracy: 0.7019
Epoch 5/10
85/85 [==============================] - 7s 87ms/step - loss: 0.1556 - accuracy: 0.9455 - val_loss: 1.0268 - val_accuracy: 0.6880
Epoch 6/10
85/85 [==============================] - 7s 86ms/step - loss: 0.0679 - accuracy: 0.9856 - val_loss: 1.0555 - val_accuracy: 0.7103
Epoch 7/10
85/85 [==============================] - 7s 85ms/step - loss: 0.0850 - accuracy: 0.9685 - val_loss: 1.3665 - val_accuracy: 0.6936
Epoch 8/10
85

#Validación

#Miscelanea

Guía conceptual: [*LeNet-5*](https://github.com/codificandobits/Clasificacion_MNIST_RedesConvolucionales_Keras/blob/master/Clasificacion_de_Digitos_Usando_Redes_Convolucionales_y_Keras.ipynb)

[Colab SL](https://colab.research.google.com/drive/1BXjv9gLsVj673dDUupsexyj010A19Y3E?usp=sharing)

[Colab NS](https://colab.research.google.com/drive/1QToJzgwbFXdzSuqXMDoH26Ph3USGgW2T?usp=sharing)

[Tutorial](https://medium.com/@hewani.co/building-a-simple-convolutional-neural-net-with-own-data-c0ee8654bacb)

[Git-hub tuto](https://github.com/rickmunene/simple-cnn-with-own-data/blob/master/Simple%20Convolutional%20Neural%20Net%20With%20Own%20Data.ipynb)

[Tutorial en Colab bien explicado](https://colab.research.google.com/drive/1a7V9Hc7ks0xxDbfCZl2_96DUGlCLuh00)

Ayudas comentadas por Néstor

[Tensor board](https://www.tensorflow.org/tensorboard?hl=es-419)

[TensorFlow Hub](https://www.tensorflow.org/hub?hl=es-419)

[Introducción al deep learning](https://www.udacity.com/course/deep-learning-pytorch--ud188)